In [1]:
import pandas as pd
from string import ascii_uppercase

def get_tidy_data(file_path):
    xls_df = pd.read_excel(file_path, skiprows=[0, 1, 3, 4])
    column_names = list(xls_df.columns)
    n_candidates = len(column_names) - 11
    candidate_numbers_names = column_names[3:(3+n_candidates)]
    column_names = ["district", "village", "office"] + candidate_numbers_names + list(ascii_uppercase[:8])
    xls_df.columns = column_names
    xls_df["district"] = xls_df["district"].apply(lambda x: x.strip())    
    imputed_district = list(xls_df["district"].values)
    for i in range(len(imputed_district)):
        if len(imputed_district[i]) == 0:
            imputed_district[i] = imputed_district[i-1]
    xls_df["district"] = imputed_district
    '''
    # 自己練習的寫法，也可以，但以運行速度來說，上面的程式碼比較快
    for i in range(1,len(xls_df["district"])):
      if len(xls_df["district"][i]) == 0:
        xls_df["district"][i] = xls_df["district"][i-1]'
    '''
    xls_df = xls_df.dropna()
    return xls_df

In [2]:
# 取得所有資料的連結
county_mayor_xlss = ["https://s3-ap-northeast-1.amazonaws.com/tw-election-2018/county-mayor/{}.xls".format(i) for i in range(701, 715)]
county_mayor_xlss += ["https://s3-ap-northeast-1.amazonaws.com/tw-election-2018/county-mayor/801.xls", "https://s3-ap-northeast-1.amazonaws.com/tw-election-2018/county-mayor/802.xls"]
city_mayor_xlss = ["https://s3-ap-northeast-1.amazonaws.com/tw-election-2018/city-mayor/{}.xls".format(i) for i in range(100, 700, 100)]
all_xls_files = city_mayor_xlss + county_mayor_xlss
len(all_xls_files)

22

In [3]:
cities = ["台北市", "新北市", "桃園市", "台中市", "台南市", "高雄市"]
counties = ["新竹縣", "苗栗縣", "彰化縣", "南投縣", "雲林縣", "嘉義縣", "屏東縣", "宜蘭縣", "花蓮縣", "台東縣", "澎湖縣", "基隆市", "新竹市", "嘉義市", "金門縣", "連江縣"]
admin_areas = cities + counties

In [4]:
df_dict = {}
for xls_file, admin_area in zip(all_xls_files, admin_areas):
    tidy_df = get_tidy_data(xls_file)
    df_dict[admin_area] = tidy_df
    print("現在正在處理{}的資料...".format(admin_area))
    print("資料外觀為：", tidy_df.shape)

現在正在處理台北市的資料...
資料外觀為： (1563, 16)
現在正在處理新北市的資料...
資料外觀為： (2446, 13)
現在正在處理桃園市的資料...
資料外觀為： (1142, 16)
現在正在處理台中市的資料...
資料外觀為： (1601, 14)
現在正在處理台南市的資料...
資料外觀為： (1322, 17)
現在正在處理高雄市的資料...
資料外觀為： (1823, 15)
現在正在處理新竹縣的資料...
資料外觀為： (404, 15)
現在正在處理苗栗縣的資料...
資料外觀為： (470, 15)
現在正在處理彰化縣的資料...
資料外觀為： (1049, 16)
現在正在處理南投縣的資料...
資料外觀為： (488, 13)
現在正在處理雲林縣的資料...
資料外觀為： (557, 15)
現在正在處理嘉義縣的資料...
資料外觀為： (486, 15)
現在正在處理屏東縣的資料...
資料外觀為： (689, 14)
現在正在處理宜蘭縣的資料...
資料外觀為： (399, 16)
現在正在處理花蓮縣的資料...
資料外觀為： (299, 14)
現在正在處理台東縣的資料...
資料外觀為： (213, 16)
現在正在處理澎湖縣的資料...
資料外觀為： (116, 18)
現在正在處理基隆市的資料...
資料外觀為： (257, 13)
現在正在處理新竹市的資料...
資料外觀為： (299, 17)
現在正在處理嘉義市的資料...
資料外觀為： (177, 15)
現在正在處理金門縣的資料...
資料外觀為： (78, 17)
現在正在處理連江縣的資料...
資料外觀為： (9, 15)


In [5]:
df_dict.keys()

dict_keys(['台北市', '新北市', '桃園市', '台中市', '台南市', '高雄市', '新竹縣', '苗栗縣', '彰化縣', '南投縣', '雲林縣', '嘉義縣', '屏東縣', '宜蘭縣', '花蓮縣', '台東縣', '澎湖縣', '基隆市', '新竹市', '嘉義市', '金門縣', '連江縣'])

In [6]:
def get_long_df(df_dict):
    mayor_votes_df = pd.DataFrame()
    for k in df_dict:
        df = df_dict[k]
        df = df.drop(labels = ["A", "B", "C", "D", "E", "F", "G", "H"], axis=1)
        candidate_infos = list(df.columns[3:])
        df = df.melt(id_vars=['district', 'village', 'office'], value_vars=candidate_infos, var_name="candidate_info", value_name='votes')
        df["admin_area"] = k
        mayor_votes_df = mayor_votes_df.append(df)
    split_candidate_info = mayor_votes_df["candidate_info"].str.split("\n", expand=True)
    mayor_votes_df["number"] = split_candidate_info[0]
    mayor_votes_df["candidate"] = split_candidate_info[1]
    mayor_votes_df["party"] = split_candidate_info[2]
    mayor_votes_df = mayor_votes_df.reset_index(drop=True)
    mayor_votes_df["party"] = mayor_votes_df["party"].apply(lambda x: "無黨籍" if x == ' ' else x)
    mayor_votes_df = mayor_votes_df[["admin_area", "district", "village", "office", "number", "party", "candidate", "votes"]]
    return mayor_votes_df

In [7]:
mayors = get_long_df(df_dict)

In [8]:
mayors.head()

,admin_area,district,village,office,number,party,candidate,votes
0,台北市,北投區,建民里,1.0,1,無黨籍,吳蕚洋,4
1,台北市,北投區,建民里,2.0,1,無黨籍,吳蕚洋,2
2,台北市,北投區,建民里,3.0,1,無黨籍,吳蕚洋,2
3,台北市,北投區,文林里,4.0,1,無黨籍,吳蕚洋,1
4,台北市,北投區,文林里,5.0,1,無黨籍,吳蕚洋,5


In [9]:
mayors.tail()

,admin_area,district,village,office,number,party,candidate,votes
62684,連江縣,北竿鄉,后沃村、橋仔村、塘岐村,5.0,4,中國國民黨,劉增應,838
62685,連江縣,北竿鄉,坂里村、白沙村、芹壁村,6.0,4,中國國民黨,劉增應,301
62686,連江縣,莒光鄉,田沃村、西坵村、青帆村,7.0,4,中國國民黨,劉增應,341
62687,連江縣,莒光鄉,大坪村、福正村,8.0,4,中國國民黨,劉增應,391
62688,連江縣,東引鄉,樂華村、中柳村,9.0,4,中國國民黨,劉增應,396


In [10]:
# 在 google colab 轉換成 csv 
mayors.to_csv("mayors.csv", index=False)

In [12]:
# 在 google colab 下載 csv
from google.colab import files

files.download("mayors.csv")

ModuleNotFoundError: No module named 'google'